# Multi-objective Robust Optimization (MORO)


This exercise demostrates the application of MORO on the lake model. In contrast to the exercises in previous weeks, we will be using a slightly more sophisticated version of the problem. For details see the MORDM assignment for this week.

## Setup MORO

Many objective robust optimization aims at finding decisions that are robust with respect to the various deeply uncertain factors. For this, MORO evalues each candidate decision over a set of scenarios. For each outcome of interest, the robusntess over this set is calculated. A MOEA is used to maximize the robustness. 

For this assignment, we will be using a domain criterion as our robustness metric. The table below lists the rules that you should use for each outcome of interest.

|Outcome of interest| threhsold  |
|-------------------|------------|
| Maximum pollution | $\leq$ 0.75|
| Inertia           | $\geq$ 0.6 |
| Reliability       | $\geq$ 0.99|   
| Utility           | $\geq$ 0.75|

**1) Implement a function for each outcome that takes a numpy array with results for the outcome of interest, and returns the robustness score**

In [41]:
import functools

def robustness(direction, threshold, data):
    if direction == SMALLER:
        return np.sum(data<=threshold)/data.shape[0]
    else:
        return np.sum(data>=threshold)/data.shape[0]

def maxp(data):
    return np.sum(data<=0.75)/data.shape[0]
    
SMALLER = 'SMALLER'
LARGER = 'LARGER'

maxp = functools.partial(robustness, SMALLER, 0.75)
inertia = functools.partial(robustness, LARGER, 0.6)
reliability = functools.partial(robustness, LARGER, 0.99)
utility = functools.partial(robustness, LARGER, 0.75)




In [63]:
from SALib.analyze import sobol

**2) Generate 4 random release policies, and evaluate them over 500 scenarios. Sample the scenarios using Monte Carlo sampling. Next evaulate your robustness function for 1, 2, 3, ... 500 scenarios for each outcome and visualize this. What can you tell about the convergernce of the robusntess metric as a function of the number of scenarios?**

In [70]:
from dps_lake_model import (lake_model)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from numpy import random
from ema_workbench import ema_logging



from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, ema_logging, Policy, perform_experiments)
from ema_workbench.em_framework import samplers
from ema_workbench import save_results, load_results

from lakemodel_function import lake_problem

#instantiate the model
lake_model = Model('lakeproblem', function=lake_problem)
lake_model.time_horizon = 100 # used to specify the number of timesteps

#specify uncertainties
lake_model.uncertainties = [RealParameter('mean', 0.01, 0.05),
                            RealParameter('stdev', 0.001, 0.005),
                            RealParameter('b', 0.1, 0.45),
                            RealParameter('q', 2.0, 4.5),
                            RealParameter('delta', 0.93, 0.99)]

# set levers, one for each time step
lake_model.levers = [RealParameter(f"l{i}", 0, 0.1) for i in 
                     range(lake_model.time_horizon)] # we use time_horizon here

#specify outcomes 
lake_model.outcomes = [ScalarOutcome('max_P'),
                       ScalarOutcome('utility'),
                       ScalarOutcome('inertia'),
                       ScalarOutcome('reliability')]

In [54]:
policies = [Policy('Random release 1',
                      **{l.name:random.uniform(0,0.1) for l in lake_model.levers}),
            Policy('Random release 2',
                       **{l.name:random.uniform(0,0.1) for l in lake_model.levers}),
            Policy('Random release 3',
                       **{l.name:random.uniform(0,0.1) for l in lake_model.levers}),
            Policy('Random release 4',
                       **{l.name:random.uniform(0,0.1) for l in lake_model.levers})
                ]

In [81]:
n_scenarios = 500
scenarios=samplers.sample_levers(lake_model,n_scenarios, sampler=samplers.MonteCarloSampler())
results = perform_experiments(lake_model, n_scenarios, policies=policies, levers_sampling=samplers.MonteCarloSampler())
experiments, outcomes = results

In [82]:
policies = experiments['policy']
for i, policy in enumerate(np.unique(policies)):
    experiments.loc[policies==policy, 'policy'] = str(i)

data = pd.DataFrame(outcomes)
data['policy'] = policies

In [83]:
data.head(20)

,max_P,utility,inertia,reliability,policy
0,0.320764,0.416655,0.626263,1.0000,Random release 1
1,3.884218,0.311133,0.626263,0.1360,Random release 1
2,0.237184,0.320027,0.626263,1.0000,Random release 1
3,2.403126,0.411375,0.626263,0.8525,Random release 1
4,6.437283,0.639811,0.626263,0.1298,Random release 1
5,5.924150,0.412557,0.626263,0.0900,Random release 1
6,6.873385,0.905246,0.626263,0.0900,Random release 1
7,0.314307,0.871820,0.626263,1.0000,Random release 1
8,0.315386,0.316154,0.626263,1.0000,Random release 1
9,0.317872,0.523840,0.626263,1.0000,Random release 1


In [84]:
potential_solutions=data.loc[(data['max_P']<=0.75) & 
                   (data['inertia']>= 0.6) & (data['reliability']>=0.99) &(data['utility']>=0.75)]

## Searching for candidate solutions
Set up the robust optimization problem using the robustness functions you have specified. Assume that you will need 50 scenarios for estimating the robustness. Use $\epsilon$-progress and hypervolume to track convergence. Solve the optimization problem. As $\epsilon$ values, you can assume 0.05 for each of the four robustness metrics.

*note: this optimization problem is computationally very expensive. Develop and test your code using a sequential evaluator, a low number of function evaluations (e.g., 200), and a low number of scenarios (e.g., 5). Once everything seems to be working replace the sequential evaluator with an multiprocessing or ipyparallel evaluator, and increase the number of nfe and scenarios*.


**Plot your $\epsilon$-progress to evaluate convergergence, and visualize the trade-offs using parallel coordinate plots**

**What does this plot tell us about the tradeoffs and conflicting objectives?**

## Re-evaluate candidate solutions under uncertainty

We have used only 50 scenarios for the optimization. Take the results and re-evaluate them over a larger set (assume 1000 scenarios). How different are your results? What does this imply for the assumption of 50 scenarios during robust optimization.

*hint: use the to_dict method on a dataframe, next generate Policy objects in a list expression by iterating over the dicts returned by the to_dict method*

## Comparison
If you have time, import your solutions found for MORDM and re-evaluate them over the same set of scnearios as used for re-evaluating the MORO results. Compare the robustness of MORDM and MORO, what do you observe?